# Welcome to IWAAN*
\**Interactive Wikipedia Article Analysis Notebooks*

This  collection of notebooks lets you analyze the evolution of the revision history of a Wikipedia article with **live data**. Among other things, it allows you to inspect **article- or editor-specific writing dynamics, productivity & conflicts.** 
The main notebooks are the current one and notebook two, with notebook three offering some additional information. 

Each Notebook can be **explored like a Web app, without interacting with the code behind it**. But you can also - if you choose to - click on **"Show solution"** after each block and work with the code directly (green buttons, comment the first line of code to use). 
 

**Please choose a small or medium-sized article for your first exploration to reduce computation times**. All data is fetched live, so please be patient and wait for the kernel to finish before you interact with the notebook.

<img src="kernel_active.png" alt="drawing" width="300"/>  

E.g., for your very first try you should simply go with our default choice, "The camp of the saints". 


---

#### Troubleshooting:

- Allow some time for the notebook to run fully before interacting with the interface controls. For articles with a long revision history, this could take minutes and the interaction with the controls will be slow.
- Very large articles ("Donald Trump", "World War 2") may lead to memory overload, depending on which Binder platform you are using. You could get more memory by shutting down some previous running notebooks.
- **All cells should run automatically** when you open the notebook and **you should not see any code**. If that is not the case, please just reload the tab in your browser. 
- After chosing a new article, please rerun the cells/modules you want to use. 
- You should not see any code when you run it for the first time. If so, please let us know by posting an issue in our Github repository: https://github.com/gesiscss/IWAAN/issues.


In [1]:
from IPython.display import display, Javascript, HTML, clear_output
from ipywidgets import widgets, Output, interact, Layout
from ipywidgets.widgets import Dropdown
from datetime import datetime

## SOME EXTENTIONS ##
#%load_ext autoreload
%reload_ext autoreload
%autoreload 2
if 'the_page' not in locals():
    import pickle
    print("Loading default data...")
    the_page = pickle.load(open("data/the_page.p",'rb'))

display(Javascript('IPython.notebook.execute_cells_below()'))

display(Javascript('Jupyter.notebook.get_cells()'))

re_hide = """
var update_input_visibility = function () {
    Jupyter.notebook.get_cells().forEach(function(cell) {
        if (cell.metadata.hide_input) {
            cell.element.find("div.input").hide();
        }
    })
};
update_input_visibility();
"""
display(Javascript(re_hide))

scroll_to_top = """
document.getElementById('notebook').scrollIntoView();
"""
display(Javascript(scroll_to_top))

Loading default data...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
%%capture
%store -r the_page

### <span style="color:green"> Import all modules </span>

In [ ]:
## Modules Imported ##

# Display
from IPython.display import display, Markdown as md, clear_output
from datetime import date
import urllib

# APIs
from wikiwho_wrapper import WikiWho
from external.wikipedia import WikipediaDV, WikipediaAPI
from external.wikimedia import WikiMediaDV, WikiMediaAPI
from external.xtools import XtoolsAPI, XtoolsDV

# Data Processing
import pickle
import pandas as pd

# Visualization tools
import qgrid
import matplotlib.pyplot as plt

# Page views timeline
from visualization.views_listener import ViewsListener

# Change actions timeline
from visualization.actions_listener import ActionsListener

# Conflicts visualization
from visualization.conflicts_listener import ConflictsListener, ConflictsActionListener
from visualization.calculator_listener import ConflictCalculatorListener

# Word cloud visualization
from visualization.wordcloud_listener import WCListener, WCActionsListener
from visualization.wordclouder import WordClouder

# Wikipedia talk pages visualization
from visualization.talks_listener import TalksListener
from visualization.topics_listener import TopicsListener

# Tokens ownership visualization
from visualization.owned_listener import OwnedListener

# Templates visualization
from visualization.templates_listener import ProtectListener, TemplateListener

# Metrics management
from metrics.conflict import ConflictManager
from metrics.token import TokensManager

# For language selection
from utils.lngselection import abbreviation, lng_listener

---

## A. Selecting a Wikipedia article

You can enter/search a Wikipedia article of your choice, or use our default example: "The Camp of the Saints" (a novel). 


In [ ]:
# the method that listens to the click event
def on_button_clicked(b):
    global wikipedia_dv
    global the_page
    # use the out widget so the output is overwritten when two or more
    # searches are performed
    with out:
        try:            
            # query wikipedia
            wikipedia_dv = WikipediaDV(WikipediaAPI(lng=abbreviation(languageSelection.value)))
            search_result = wikipedia_dv.search_page(searchTerm.value)
            the_page = wikipedia_dv.get_page(search_result)
            %store the_page
            clear_output()
            display(md(f"The following page was found: **{the_page['title']}**"))
            display(md(f"Page id: **{the_page['page_id']}**"))
            url = f"{wikipedia_dv.api.base}action=query&titles={urllib.parse.quote_plus(the_page['title'])}&format=json"
            display(md(f"Metadata can be found in:"))
            print(url)
            #display(Javascript('Jupyter.notebook.execute_cells([8])'))

        except:
            clear_output()
            display(md(f'The page title *"{searchTerm.value}"* was not found'))
            #display(Javascript('Jupyter.notebook.execute_cells([8])'))

# Language selection.
languageSelection = Dropdown(options=['English', 'Deutsch', 'Español', 'Türkçe', 'Euskara'], value='English', description='Language:')

out_main = Output()
display(out_main)
with out_main:
    # by default display the last search
    try:
        searchTerm = widgets.Text(the_page['title'], description='Page title:')
    except:
        searchTerm = widgets.Text("The Camp of the Saints", description='Page title:')

    # Update selected language
    initial_select = widgets.interactive(lng_listener, lng=languageSelection, search_term=searchTerm)
    display(initial_select.children[0])
    display(initial_select.children[1])

    # create and display the button    
    button = widgets.Button(description="Search")
    example = md("e.g. *The Camp of the Saints*")
    display(example, button)

# the output widget is used to remove the output after the search field
out = Output()
display(out)

# set the event
button.on_click(on_button_clicked)

# trigger the event with the default value
on_button_clicked(button)

<span style="color: #626262"> Try yourself! This is the code executed when  you click the 'Search' button: </span>

In [ ]:
%%script false --no-raise-error

### IMPORTANT NOTE: COMMENT THE ABOVE LINE TO EXECUTE THE CELL ###

### --------------------------------------------------------------------- ###
### TRY YOURSELF! THIS IS WHAT WILL HAPPEN WHEN YOU CLICK 'Search' BUTTON ###
### --------------------------------------------------------------------- ###

## This is the default data and used for initialization ##
the_page = pickle.load(open("data/the_page.p",'rb'))  # global
title_default = the_page['title']
print('The pre-filled value for the title:', title_default)

## The search term you have input ##

# this was extracted from the previous search box, searchTerm, which is built by
# e.g. searchTerm = widgets.Text(the_page['title'], description='Page title:'), you could
# know more about it through https://ipywidgets.readthedocs.io/en/latest/examples/Widget%20List.html
search_language = languageSelection.value
language_for_api = abbreviation(search_language)

search_value = searchTerm.value
print('The language you use now:', search_language)
print('The value you input in search box:', search_value)

# of course you could also update the value here, like in the "Page title:" box above.
#search_value = 'Matrix Completion' 

## query wikipedia using WikiWho API, more details please see:                ##
## https://github.com/gesiscss/wikiwho_demo/blob/master/external/wikipedia.py ##
## https://github.com/gesiscss/wikiwho_demo/blob/master/external/api.py       ##
wikipedia_dv = WikipediaDV(WikipediaAPI(lng=language_for_api)) # create an instance
result_after_search = wikipedia_dv.search_page(search_value)

print("The following page was found:", result_after_search)

## Get page id through get_page() method ##
the_page = wikipedia_dv.get_page(result_after_search)  # global
page_id = the_page['page_id']
print('Page id:', page_id)

## Metadata ##
url = f"{wikipedia_dv.api.base}action=query&titles={urllib.parse.quote_plus(result_after_search)}&format=json"
print('Metadata can be found in:', url)

---

**Click on "Refresh All Variables" to reload all following cells with the selected article. (may take a some time)** 

Alternatively, reload each module individually via their respective buttons.

In [ ]:
def restart_and_run():
    display(HTML(
        '''
            <script>
                code_show = false;
                function restart_run_all(){
                    IPython.notebook.kernel.restart();
                    setTimeout(function(){
                        IPython.notebook.execute_all_cells();
                    })
                }
            </script>
            <button onclick="restart_run_all()">Refresh All Variables</button>
        '''
    ))
restart_and_run()

---

## B. General Statistics

Thanks to the Xtools API **(1)**, we can obtain general statistics related to the articlee. This information is relevant to select an appropiate article to analyze. 

Our default example contains less than 1000 revisions (as of April, 2021), making it a good first case for our analysis. Articles such as *"Chicago"*, *"Barack Obama"* and *"Climate Change"* can have over 20K revisions, and therefore the processing will be slow and even fail due to lack of memory on the system running the notebooks (e.g.[notebooks.gesis.org](https://notebooks.gesis.org)).

Wikipedia offers a [list of the pages with most revisions](https://en.wikipedia.org/wiki/Wikipedia:Database_reports/Pages_with_the_most_revisions); **we recommend to avoid such articles for a smooth first experience with IWAAN**. Once you are confortable with our notebooks, you can tackle more complex articles.

In [ ]:
def xtools_on_click(b):
    with out_xtools:
        clear_output()
        xtools_api = XtoolsAPI(lng=abbreviation(languageSelection.value))
        xtools_dv = XtoolsDV(xtools_api)
        page_info = xtools_dv.get_page_info(the_page['title'])
        page_info['assessment'] = page_info['assessment']['value'] if type(page_info['assessment']) != bool else page_info['assessment']

        page_info = page_info.to_frame('value').rename(index={
            'project': 'Project name',
            'page': 'Page name',
            'watchers': 'Watchers (2)',    'pageviews': f"Page Views (per {page_info['pageviews_offset']} days)",
            'revisions': 'Revisions',
            'editors': 'Editors',
            'author': 'Creator of the page',
            'created_at': 'Creation Date',
            'created_rev_id': 'Creation revision id',
            'modified_at': 'Last modified',
            'last_edit_id': 'Last revision id',
            'assessment': 'Content Assessment (3)',
        }).drop(index = ['pageviews_offset', 'author_editcount', 'secs_since_last_edit','elapsed_time'])
        
        display(md(f"***Page: {the_page['title']} ({abbreviation(languageSelection.value).upper()})***"))
        url = f"{xtools_dv.api.base}page/articleinfo/{xtools_dv.api.project}/" + urllib.parse.quote(the_page['title'])
        display(md(f"Metadata can be found in"))
        print(url)
        display(page_info)
        #display(Javascript('Jupyter.notebook.execute_cells([14])'))           
        #display(Javascript('Jupyter.notebook.execute_cells([18])'))
        

# create and display the button    
button = widgets.Button(description="Get Page Info")
display(button)

# the output widget is used to remove the output after the search field
out_xtools = Output()
display(out_xtools)

# set the event
button.on_click(xtools_on_click)

# trigger the event with the default value
xtools_on_click(button)

**(1)** *A community-built service for article statistics at xtools.wmflabs.org*<br>
**(2)** *Users that added this page to their watchlist*<br>
**(3)** *See [Wikipedia Content Assessment](https://en.wikipedia.org/wiki/Wikipedia:Content_assessment)*


<span style="color: #626262"> Try yourself! This is the code executed when  you click the 'Get Page Info' button: </span>

In [ ]:
%%script false --no-raise-error

### IMPORTANT NOTE: COMMENT THE ABOVE LINE TO EXECUTE THE CELL ###

### ----------------------------------------------------------------------------- ###
### TRY YOURSELF! THIS IS WHAT WILL HAPPEN WHEN YOU CLICK 'Get Page Info' BUTTON  ###
### ----------------------------------------------------------------------------- ###

## Define a Xtools instance, more details see:                             ##
## https://github.com/gesiscss/wikiwho_demo/blob/master/external/xtools.py ##
## https://github.com/gesiscss/wikiwho_demo/blob/master/external/api.py    ##
xtools_api = XtoolsAPI(lng=abbreviation(languageSelection.value))
xtools_dv = XtoolsDV(xtools_api)
print('Provided through the Xtools API (1)')

## The page you are insterested in ##
print('The page that was found:', the_page['title'], f'({abbreviation(languageSelection.value).upper()})')

## Get the page info through Xtools method get_page_info() ##
page_info = xtools_dv.get_page_info(the_page['title'])

## Metadata ##
url = f"{xtools_dv.api.base}page/articleinfo/{xtools_dv.api.project}/" + urllib.parse.quote(the_page['title'])
print("Metadata can be found in:", url)

## Use a dictionary to construct a pd.DataFrame to present the general info from Xtools ##
dict_for_df = {
    'Project name': page_info['project'], 'Page name': page_info['page'], 'Watchers(2)': page_info['watchers'],
    'Page Views (per 30 days)': page_info['pageviews'], 'Revisions': page_info['revisions'], 
    'Editors': page_info['editors'], 'minor_edits': page_info['minor_edits'], 'Creator of the page': page_info['author'],
    'Creation Date': page_info['created_at'], 'Creation revision id': page_info['created_at'],
    'Last modified': page_info['modified_at'], 'Last revision id': page_info['last_edit_id'],
    'Content Assessment (3)': page_info['assessment']['value'] if type(page_info['assessment']) != bool else page_info['assessment']
}

df_info = pd.DataFrame.from_dict(dict_for_df, orient='index', columns=['value'])
display(df_info)

## Some footnotes ##
display(md('**(1)** *A community-built service for article statistics at xtools.wmflabs.org*<br>'
           '**(2)** *Users that added this page to their watchlist*<br>'
           '**(3)** *See [Wikipedia Content Assessment](https://en.wikipedia.org/wiki/Wikipedia:Content_assessment)*'))

---

## C. Page Views

We use the Wikimedia Pageviews API to display the page views of an article, which can be helpful when assesing the impact of changes in Wikipedia articles. For example, it is possible that a period with a lot of changes might due to internal politics of Wikipedia rather than an increase of interest on the topic by the general population.

In [ ]:
def pageviews_button(b):
    with out_pageviews:
        clear_output()
        
        display(md(f"***Page: {the_page['title']} ({abbreviation(languageSelection.value).upper()})***"))
        # Query request
        wikimedia_api = WikiMediaAPI(lng=abbreviation(languageSelection.value))
        wikimedia_dv = WikiMediaDV(wikimedia_api)
        views = wikimedia_dv.get_pageviews(the_page['title'], 'daily')
        
        # Visualization
        listener = ViewsListener(views)
        inter_func = interact(listener.listen, 
                         begin=Dropdown(options=views.timestamp),
                         end=Dropdown(options=views.timestamp.sort_values(ascending=False)),
                         granularity=Dropdown(options=['Yearly', 'Monthly', 'Weekly', 'Daily'], value='Monthly'))

        # raw data url
        start = 19900101
        today = datetime.today().strftime("%Y%m%d")
        end = int(today)
        article_name = urllib.parse.quote(the_page['title'])
        granularity = 'daily'

        url = (f'{wikimedia_dv.api.base}metrics/pageviews/per-article/{wikimedia_dv.api.project}/'
                f'all-access/all-agents/{article_name}/{granularity}/{start}/{end}')
        display(md(f"Metadata can be found in:"))
        print(url)
                   
        # The df_plotted keeps a reference to the plotted data above
        pageviews_agg = listener.df_plotted['views'].agg({
                            'Total views': sum,
                            'Max views period': max,
                            'Min views period': min,
                            'Average views': min,}).to_frame('Value')
        display(pageviews_agg)
        
        

# create and display the button    
button = widgets.Button(description="Get Pageviews", layout=Layout(width='150px'))
display(button)

# the output widget is used to remove the output after the search field
out_pageviews = Output()
display(out_pageviews)

# set the event
button.on_click(pageviews_button)

# trigger the event with the default value
pageviews_button(button)

<span style="color: #626262"> Try yourself! This is the code executed when  you click the  'Get Pageviews' button: </span>

In [ ]:
%%script false --no-raise-error

### IMPORTANT NOTE: COMMENT THE ABOVE LINE TO EXECUTE THE CELL ###

### ----------------------------------------------------------------------------- ###
### TRY YOURSELF! THIS IS WHAT WILL HAPPEN WHEN YOU CLICK 'Get Pageviews' BUTTON  ###
### ----------------------------------------------------------------------------- ###

## define a WikiMediaAPI instance, more details see:                          ##
## https://github.com/gesiscss/wikiwho_demo/blob/master/external/wikimedia.py ##
## https://github.com/gesiscss/wikiwho_demo/blob/master/external/api.py       ##


wikimedia_api = WikiMediaAPI(lng=abbreviation(languageSelection.value))
wikimedia_dv = WikiMediaDV(wikimedia_api)

## Page of insterest ##
print('The page that was found:', the_page['title'], f'({abbreviation(languageSelection.value).upper()})')

## get pageview counts for the article, more details see:                     ##
## https://github.com/gesiscss/wikiwho_demo/blob/master/external/wikimedia.py ##

views = wikimedia_dv.get_pageviews(the_page['title'], 'daily')

## Visualization, core visual code lies in ViewsListener, then the interact function    ##
## make it interactive, mode details see:                                               ##
## https://github.com/gesiscss/wikiwho_demo/blob/master/visualization/views_listener.py ##
## https://ipywidgets.readthedocs.io/en/latest/examples/Using%20Interact.html           ##

# Create a ViewListener instance with the page views counts
listener = ViewsListener(views)

# You could customize begin/end dates and granularity to generate different graphs
# e.g. begin='20160809', end='20191020', granularity='Monthly'
begin='20150701'
end='20210314'
granularity='Monthly' # 'Yearly', 'Monthly', 'Weekly' ,'Daily' 

# Metadata
url = (f'{wikimedia_dv.api.base}metrics/pageviews/per-article/'
       f"{wikimedia_dv.api.project}/all-access/all-agents/{urllib.parse.quote(the_page['title'])}/{granularity.lower()}"
       f"/{int(begin)}/{int(end)}")
print('Metadata can be found in:', url)
print('(Note that: the time unit for the response data only supports granularity for "daily" and "monthly")\n')

# Time range you have selected
print('Time range you have selected:')
print('Start date:', begin)
print('End date:', end)
print('Granularity:', granularity)

# Visulization
listener.listen(begin, end, granularity)

# Pageviews aggregation data. Use the attribute "df_plotted".
pageviews_agg = listener.df_plotted['views'].agg({
                    'Total views': sum,
                    'Max views period': max,
                    'Min views period': min,
                    'Average views': min,}).to_frame('Value')

print('Total views of this page:', pageviews_agg['Value']['Total views'])
print('Max views during the selected period:', pageviews_agg['Value']['Max views period'])
print('Min views during the selected period:', pageviews_agg['Value']['Min views period'])
print('Average views during the selected period:', pageviews_agg['Value']['Average views'])

---

# D. Templates and Protection

In this step, all the [WikiWho tokens](https://www.wikiwho.net/) and associated revisions are downloaded from the [WikiWho API](https://www.wikiwho.net/). Depending on the article, this might take some time, but, once finished, it will display the period of times in which the article was on any of the following statuses:

- **Semi-page edit protection**: all phases in the article’s life where edits or moves where only allowed for registered users
- **Full-page edit protection**: all phases in the article’s life where edits or moves where only allowed for rusers with special rights, such  as  admins
- **Good article**: as described by Wikipedia guidelines for [good articles](https://en.wikipedia.org/wiki/Wikipedia:Good_articles)
- **Featured article**: as described by Wikipedia guidelines for [featured articles](https://en.wikipedia.org/wiki/Wikipedia:Featured_articles)
- **Special templates**: special templates posted by editors indicating disputes, as well as bias and neutrality issues

Similar to the page views, this information puts the edits, conflicts and discussions in the the following notebook in   context.

In [ ]:
def template_button(b):
    global token_source
    global new_template
    global new_protect
    global token_source
    with template_out:
        clear_output()
        
        # WikiWho API.
        wikiwho = WikiWho(lng=abbreviation(languageSelection.value))
        display(md("Downloading all_content from the WikiWhoApi..."))
        content = wikiwho.dv.all_content(the_page['page_id'])
        display(md("Downloading revisions from the WikiWhoApi..."))
        revisions = wikiwho.dv.rev_ids_of_article(the_page['page_id'])
        clear_output()
        
        # Wikipedia API
        pp_log = wikipedia_dv.get_protection(the_page['title'])

        # Use ConflictManager to join content and revision tables.
        cm = ConflictManager(content,
                             revisions, 
                             lng=abbreviation(languageSelection.value), 
                             include_stopwords=True)
        cm.calculate()
        clear_output()
        token_source = cm.all_actions.copy()
        
        display(md(f"***Page: {the_page['title']} ({abbreviation(languageSelection.value).upper()})***"))

        # Templateslistener
        new_protect = ProtectListener(pp_log, lng=abbreviation(languageSelection.value))
        
        display(md("Analysing protection data..."))
        plot_protect = [new_protect.get_protect(level)[1] for level in ["semi_edit", "semi_move", "fully_edit", "fully_move", "unknown"]]
        plot_protect = pd.concat(plot_protect)

        new_template = TemplateListener(token_source, plot_protect, lng=abbreviation(languageSelection.value),
                                       wikipediadv_api=wikipedia_dv, page=the_page)
        new_template.listen()
        
        
# create and display the button    
button = widgets.Button(description="Get Templates & Protection", layout=Layout(width='200px'))
display(button)

# the output widget is used to remove the output after the search field
template_out = Output()
display(template_out)

# set the event
button.on_click(template_button)

# trigger the event with the default value
template_button(button)


Now that we have a base understanding of the historical context of the article, we will explore  specific kinds of changes that have affected the articles, and the editors involved in its evolution. 

Click below to go to the next notebook . This notebook stays open in its own tab and you can always come back to it. Or simply enter another article name to start the process over with a new article. 

[Go to next workbook](./2_Page_analysis.ipynb)

In [ ]:
# from utils.notebooks import get_next_notebook
# display(HTML(f'<a href="{get_next_notebook()}" target="_blank">Go to next workbook</a>'))



scroll_to_top = """
document.getElementById('notebook').scrollIntoView();
"""
display(Javascript(scroll_to_top))